In [12]:
from dms_stan.model.stan.stan_results import SampleResults

In [17]:
test = SampleResults.from_disk(
    "/home/bwittmann/GitRepos/DMSStan/tmp/trpb/libA/trpb_libA_exponential_exponential_diagnosed.nc",
    skip_fit=True,
)

In [21]:
test.inference_obj.variable_diagnostic_stats.total_raw_abundance_tg0.values

array([[1.01000e+00, 1.01000e+00],
       [1.17816e+03, 1.17974e+03],
       [0.00000e+00, 0.00000e+00],
       [0.00000e+00, 0.00000e+00],
       [2.10216e+03, 2.11048e+03]])